# Install the required packages

In [1]:
!pip install wikipedia
!pip install nltk
!pip install pyspellchecker
!pip install networkx
!pip install dgl

In [2]:
!apt update
!apt install enchant --fix-missing
!apt install -qq enchant
!pip install pyenchant

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


# Import the required libraries

In [1]:
import os
import wikipedia
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from spellchecker import SpellChecker
import enchant
import dgl
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import pickle
import torch
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv
from torch.utils.data.sampler import SubsetRandomSampler
from dgl.dataloading import GraphDataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Set the embedding size and GPU device

In [2]:
EMBEDDING_DIMENSION = 64
device = torch.device('cuda')
torch.cuda.is_available()

True

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/admin1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/admin1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Create a custom Wikipedia corpus

In [53]:
# The variable "keyword" decides what type of articles to pull from wikipedia
# We tested our approach on the following keywords: sports, celebrity, music

keyword = "music"
corpus_file = "data/" + keyword + "_corpus.pk"
corpus = ""

if os.path.isfile(corpus_file):
    corpus = pickle.load(open(corpus_file, "rb"))
else:

    search_results = wikipedia.search(keyword, results=1000)

    for result in search_results:
        try:
            summary = wikipedia.summary(result)
            corpus += summary
        except:
            print("Some Error occured for keyword : ", result)
        finally:
            pass
    pickle.dump(corpus, open(corpus_file, "wb"))

Some Error occured for keyword :  Music


/home/admin1/anaconda3/envs/tf_gpu_env/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/admin1/anaconda3/envs/tf_gpu_env/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Some Error occured for keyword :  Music (disambiguation)
Some Error occured for keyword :  Pop music
Some Error occured for keyword :  Sony Music
Some Error occured for keyword :  K-pop
Some Error occured for keyword :  Musical instrument
Some Error occured for keyword :  Composer
Some Error occured for keyword :  Record producer
Some Error occured for keyword :  Garage
Some Error occured for keyword :  20th-century music
Some Error occured for keyword :  Pop rock
Some Error occured for keyword :  Rebel Music
Some Error occured for keyword :  Jazz
Some Error occured for keyword :  Music group (disambiguation)
Some Error occured for keyword :  American Music Awards of 2002
Some Error occured for keyword :  Mode (music)
Some Error occured for keyword :  Roots music
Some Error occured for keyword :  English music
Some Error occured for keyword :  Musical notation
Some Error occured for keyword :  Music player
Some Error occured for keyword :  Techno
Some Error occured for keyword :  Roxy 

# Pre-process the text corpus

In [7]:
def remove_whitespace(text):
    return  " ".join(text.split())
 
def remove_stopwords(text_tokens):
    result = []
    en_stopwords = stopwords.words('english')
    
    for token in text_tokens:
        if token not in en_stopwords:
            result.append(token)

    return result

def remove_punctuations(text):
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

In [8]:
def pre_process_text(text_corpus):
    #Step 1: Convert every word to lower case to avoid any ambiguity
    lower_text_corpus = text_corpus.lower()
    
    #Step 2: Remove extra whitespaces
    lower_text_corpus = remove_whitespace(lower_text_corpus)
    
    #Step 3: Tokenize in word level
    tokens = word_tokenize(lower_text_corpus)
    
    #Step 4: Remove the stopwords
#     tokens = remove_stopwords(tokens)
    
    #Step 5: Remove punctuations
    tokens = remove_punctuations(tokens)
    return " ".join(tokens)
    

In [56]:
corpus[:1000]

'"Music! Music! Music! (Put Another Nickel In)" is a popular song written by Stephen Weiss and Bernie Baum and published in 1950."Music! Music! Music! (Put Another Nickel In)" is a popular song written by Stephen Weiss and Bernie Baum and published in 1950.Country (also called country and western) is a genre of popular music that originated with blues, church music such as Southern gospel and spirituals, old-time, and American folk music forms including Appalachian, Cajun, Creole, Hawaiian, and the cowboy Western music styles of New Mexico, Red Dirt, Tejano, and Texas country. Its popularized roots originate in the Southern and Southwestern United States of the early 1920s.\nCountry music often consists of ballads and honky-tonk dance tunes with generally simple form, folk lyrics, and harmonies often accompanied by string instruments such as electric and acoustic guitars, steel guitars (such as pedal steels and dobros), banjos, and fiddles as well as harmonicas. Blues modes have been u

In [57]:
processed_corpus = pre_process_text(corpus)

In [58]:
processed_corpus[:1000]

'music music music put another nickel in is a popular song written by stephen weiss and bernie baum and published in 1950 music music music put another nickel in is a popular song written by stephen weiss and bernie baum and published in 1950 country also called country and western is a genre of popular music that originated with blues church music such as southern gospel and spirituals old time and american folk music forms including appalachian cajun creole hawaiian and the cowboy western music styles of new mexico red dirt tejano and texas country its popularized roots originate in the southern and southwestern united states of the early 1920s country music often consists of ballads and honky tonk dance tunes with generally simple form folk lyrics and harmonies often accompanied by string instruments such as electric and acoustic guitars steel guitars such as pedal steels and dobros banjos and fiddles as well as harmonicas blues modes have been used extensively throughout its record

In [59]:
len(processed_corpus.split(" "))

76600

# Construct lookup for word-to-index and index-to-word

In [60]:
# word_count_map = dict()
word_index = dict()
index_word = dict()
processed_word_tokens = processed_corpus.split(" ")
unique_words = list(set(processed_word_tokens))

for idx, word in enumerate(unique_words):
    word_index[word] = idx
    index_word[idx] = word

In [61]:
len(word_index)

9344

# Construct a single large graph for the pre-processed corpus

In [62]:
src = []
dst = []

for idx, word in enumerate(processed_word_tokens):
    if idx < (len(processed_word_tokens)-1):
        src.append(word_index[word])
        dst.append(word_index[processed_word_tokens[idx+1]])

In [63]:
dgl_graph = dgl.DGLGraph()
dgl_graph.add_edges(src, dst)

In [64]:
print('We have %d nodes.' % dgl_graph.number_of_nodes())
print('We have %d edges.' % dgl_graph.number_of_edges())

We have 9344 nodes.
We have 76599 edges.


# Initialize the vector embeddings for each node in the graph

In [65]:
embedding_layer = torch.nn.Embedding(len(unique_words), EMBEDDING_DIMENSION)

In [66]:
input_emb = torch.LongTensor([range(0,len(unique_words),1)])
node_embeddings = None
with torch.no_grad():
    node_embeddings = embedding_layer(input_emb).squeeze()

In [67]:
node_embeddings.shape

torch.Size([9344, 64])

# Define Graph dataset as a DGLDataset

In [68]:
class WikipediaCorpusGraphDataset(dgl.data.DGLDataset):
    def __init__(self, src, dst, features):
        self.src = src
        self.dst = dst
        self.features = features
        self.graph = None
        super().__init__(name="wikipedia_corpus")
    
    def process(self):
        self.graph = dgl.graph((self.src, self.dst))
        self.graph.ndata['feat'] = self.features
        
    def edges(self):
        return self.src, self.dst
        

# Train Node Embeddings as Edge Prediction on Graph Dataset

In [69]:
graph_dataset = WikipediaCorpusGraphDataset(src, dst, node_embeddings)

In [70]:
graph = graph_dataset.graph
graph.number_of_edges()

76599

In [71]:
u, v = graph.edges()

eids = np.arange(graph.number_of_edges())
eids = np.random.permutation(eids)

test_size = int(len(eids) * 0.5)
train_size = graph.number_of_edges() - test_size

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

In [72]:
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(graph.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), graph.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [73]:
train_g = dgl.remove_edges(graph, eids[:test_size])

In [74]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=graph.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=graph.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=graph.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=graph.number_of_nodes())

# Define the GCN model

In [75]:
class GCNStack(nn.Module):
    def __init__(self, in_feats, h_feats, num_layers=2, dropout=0.25):
        super(GCNStack, self).__init__()
        self.convs = nn.ModuleList()
        self.convs.append(self.build_conv_model(in_feats, h_feats))
        
        for i in range(num_layers-1):
            self.convs.append(self.build_conv_model(h_feats, h_feats))
            
        self.num_layers = len(self.convs)
        self.lns = nn.ModuleList()
        for i in range(self.num_layers-1):
            self.lns.append(nn.LayerNorm(h_feats))
        
        self.dropout = dropout
           
    def build_conv_model(self, input_dim, hidden_dim):
        return GraphConv(input_dim, hidden_dim, allow_zero_in_degree=True)
    
    def forward(self, graph, input_features):
        x = input_features
        embeddings = None
        
        for i, conv_layer in enumerate(self.convs):
            x = conv_layer(graph, x)
            x = F.relu(x)
            embeddings = x
            x = F.dropout(x, p=self.dropout, training = self.training)
            
            if not i == self.num_layers-1:
                x = self.lns[i](x)
            
        return embeddings
    
    def loss(self, pred, label):
        return F.nll_loss(pred, label)
        

In [76]:
criterion = torch.nn.BCELoss()

In [77]:
def compute_loss(pos_score, neg_score):
#     print("Compute LOSS :: Positive Scores :: ", pos_score)
#     print("Compute LOSS :: Negative Scores :: ", neg_score)
    scores = torch.cat([pos_score.float(), neg_score.float()])
    pos_labels = []
    neg_labels = []
    for i in range(pos_score.shape[0]):
#         pos_labels.append([1,0])
        pos_labels.append([1])
    for i in range(neg_score.shape[0]):
#         neg_labels.append([0,1])
        neg_labels.append([0])
    labels = torch.cat([torch.Tensor(pos_labels), torch.Tensor(neg_labels)]).to(device)
#     print("Scores = ", scores)
#     print("Labels = ", labels)
    
#     return F.binary_cross_entropy_with_logits(scores, labels)
    return criterion(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score.float(), neg_score.float()])
    pos_labels = []
    neg_labels = []
    for i in range(pos_score.shape[0]):
#         pos_labels.append([1,0])
        pos_labels.append([1])
    for i in range(neg_score.shape[0]):
#         neg_labels.append([0,1])
        neg_labels.append([0])
    labels = torch.cat([torch.Tensor(pos_labels), torch.Tensor(neg_labels)]).to(device)
    return roc_auc_score(labels.detach().cpu().numpy(), scores.detach().cpu().numpy())

def compute_ap(pos_score, neg_score):
    scores = torch.cat([pos_score.float(), neg_score.float()])
    pos_labels = []
    neg_labels = []
    for i in range(pos_score.shape[0]):
#         pos_labels.append([1,0])
        pos_labels.append([1])
    for i in range(neg_score.shape[0]):
#         neg_labels.append([0,1])
        neg_labels.append([0])
    labels = torch.cat([torch.Tensor(pos_labels), torch.Tensor(neg_labels)]).to(device)
    return average_precision_score(labels.detach().cpu().numpy(), scores.detach().cpu().numpy())

In [78]:
def train_edge_classification(nc_model,linear_pred_edges, dgl_dataset, epochs=100):
    optimizer = torch.optim.Adam(nc_model.parameters(), lr=0.01)
    optimizer_linear_pred = torch.optim.Adam(linear_pred_edges[0].parameters(), lr=0.01)
    
    nc_model = nc_model.to(device)
    linear_pred_edges = linear_pred_edges.to(device)
    
    nc_model.train()
    linear_pred_edges.train()
    
    graph = dgl_dataset.graph.to(device)
    
    best_val_acc = 0
    best_test_acc = 0

    features = graph.ndata['feat']
    node_embeddings = None
    edges_src, edges_dst = graph.edges()
    train_pos_edges_src, train_pos_edges_dst = train_pos_g.edges()
    train_neg_edges_src, train_neg_edges_dst = train_neg_g.edges()
    
    test_pos_edges_src, test_pos_edges_dst = test_pos_g.edges()
    test_neg_edges_src, test_neg_edges_dst = test_neg_g.edges()
    
    for e in tqdm(range(epochs)):
        num_correct = 0
        num_tests = 0
        
        node_embeddings = nc_model(graph, features)

        edge_embeddings_pos = (node_embeddings[train_pos_edges_src] * node_embeddings[train_pos_edges_dst])/2
        edge_embeddings_neg = (node_embeddings[train_neg_edges_src] * node_embeddings[train_neg_edges_dst])/2
        
        test_edge_embeddings_pos = (node_embeddings[test_pos_edges_src] * node_embeddings[test_pos_edges_dst])/2
        test_edge_embeddings_neg = (node_embeddings[test_neg_edges_src] * node_embeddings[test_neg_edges_dst])/2
    
        pred_edges_pos = linear_pred_edges(edge_embeddings_pos).to(device)
        pred_edges_neg = linear_pred_edges(edge_embeddings_neg).to(device)

        loss = compute_loss(pred_edges_pos, pred_edges_neg)
        
        optimizer.zero_grad()
        optimizer_linear_pred.zero_grad()

        loss.backward()
        optimizer.step()
        optimizer_linear_pred.step()
        
        if e % 10 == 0:
            print('In epoch {}, loss: {}'.format(e, loss))
            print('AUC Score -> ', compute_auc(pred_edges_pos, pred_edges_neg))
            print('AP Score  -> ', compute_ap(pred_edges_pos, pred_edges_neg))

    with torch.no_grad():
        pos_score = linear_pred_edges(test_edge_embeddings_pos).to(device)
        neg_score = linear_pred_edges(test_edge_embeddings_neg).to(device)
        print('Test AUC = ', compute_auc(pos_score, neg_score))
        print('Test AP  = ', compute_ap(pos_score, neg_score))
        
    return node_embeddings

# Train Edge Classification / Link Prediction

In [79]:
num_feats = graph_dataset.graph.ndata['feat'].shape[1]
nc_model = GCNStack(num_feats, EMBEDDING_DIMENSION, num_layers=2)

In [80]:
linear_pred_edges = torch.nn.Sequential(torch.nn.Linear(EMBEDDING_DIMENSION,1), torch.nn.Sigmoid())
word_embeddings = train_edge_classification(nc_model,linear_pred_edges, graph_dataset, epochs=200)

  0%|                                                   | 0/200 [00:00<?, ?it/s]

In epoch 0, loss: 0.6822222471237183
AUC Score ->  0.6762261103422889
AP Score  ->  0.7369128088844534


  4%|█▉                                         | 9/200 [00:01<00:19,  9.56it/s]

In epoch 10, loss: 0.49734437465667725
AUC Score ->  0.9245177555917621


  6%|██▋                                       | 13/200 [00:02<00:21,  8.81it/s]

...

In epoch 190, loss: 0.20679619908332825
AUC Score ->  0.9745665574787477


 96%|███████████████████████████████████████▎ | 192/200 [00:20<00:01,  7.56it/s]

AP Score  ->  0.9772354546070432


100%|█████████████████████████████████████████| 200/200 [00:21<00:00,  9.32it/s]


Test AUC =  0.9645492760261468
Test AP  =  0.9699718507665834


In [81]:
word_embeddings.shape

torch.Size([9344, 64])

In [82]:
word_embeddings[0]

tensor([0.7975, 0.0000, 0.4177, 0.6892, 1.1205, 0.0000, 0.9303, 1.1190, 2.3185,
        0.0000, 0.8899, 0.0000, 0.0000, 0.0000, 0.1875, 0.0000, 0.6177, 0.0000,
        0.0000, 0.7564, 0.9341, 0.0000, 1.9227, 0.0000, 1.3086, 1.3907, 0.0000,
        0.0000, 1.3917, 0.3986, 0.0000, 0.0000, 0.6023, 0.9117, 0.0000, 0.9524,
        0.6193, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9112, 0.6968, 1.1294,
        0.4323, 0.0000, 0.0000, 0.8983, 0.9505, 0.0000, 0.4560, 0.1150, 0.7290,
        0.1269, 0.2383, 0.0000, 0.0942, 0.0000, 1.6153, 0.8349, 0.0000, 0.1989,
        0.9258], device='cuda:0', grad_fn=<SelectBackward0>)

# Use the Word Embeddings from GCN for LSTM

### 1. Create n-gram sequences from the original corpus

In [83]:
one_gram_sequences = []
two_gram_sequences = []
three_gram_sequences = []
five_gram_sequences = []
ten_gram_sequences = []
processed_corpus_tokens = processed_corpus.split(" ")

corpus_with_indices = [word_index[w] for w in processed_corpus_tokens]

for idx, word in enumerate(corpus_with_indices):
    if idx+11 < len(processed_corpus_tokens):
        one_gram_sequences.append(processed_corpus_tokens[idx:idx+2])
        two_gram_sequences.append(processed_corpus_tokens[idx:idx+3])
        three_gram_sequences.append(processed_corpus_tokens[idx:idx+4])
        five_gram_sequences.append(processed_corpus_tokens[idx:idx+6])
        ten_gram_sequences.append(processed_corpus_tokens[idx:idx+11])

In [84]:
zero_array = np.zeros((EMBEDDING_DIMENSION))
# zero_array

In [85]:
zero_array = np.zeros((EMBEDDING_DIMENSION))
def word_seq_to_padded_embeddings(word_seq, word_index, max_seq_length=3):
    index_seq = []
    for seq in word_seq:
        embedding_list = []
        additional_zero_pads = max_seq_length - len(seq)
        for word in seq:
            idx = word_index[word]
            embedding_list.append(word_embeddings[idx].cpu().detach().numpy())
        for i in range(additional_zero_pads):
            embedding_list.append(zero_array)
        index_seq.append(np.array(embedding_list))
    return index_seq
    

In [86]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [87]:
from torch.nn.utils.rnn import pad_sequence

In [88]:
X = []
Y = []
X.extend(word_seq_to_padded_embeddings(np.array(one_gram_sequences)[:,:-1], word_index, 10))
y_one_gram_indices = [word_index[word] for word in np.array(one_gram_sequences)[:,-1]]
y_one_gram_onehot = to_categorical(y_one_gram_indices, len(unique_words))
Y.extend(y_one_gram_onehot)

X.extend(word_seq_to_padded_embeddings(np.array(two_gram_sequences)[:,:-1], word_index, 10))
y_two_gram_indices = [word_index[word] for word in np.array(two_gram_sequences)[:,-1]]
y_two_gram_onehot = to_categorical(y_two_gram_indices, len(unique_words))
Y.extend(y_two_gram_onehot)

X.extend(word_seq_to_padded_embeddings(np.array(three_gram_sequences)[:,:-1], word_index, 10))
y_three_gram_indices = [word_index[word] for word in np.array(three_gram_sequences)[:,-1]]
y_three_gram_onehot = to_categorical(y_three_gram_indices, len(unique_words))
Y.extend(y_three_gram_onehot)

X.extend(word_seq_to_padded_embeddings(np.array(five_gram_sequences)[:,:-1], word_index, 10))
y_five_gram_indices = [word_index[word] for word in np.array(five_gram_sequences)[:,-1]]
y_five_gram_onehot = to_categorical(y_five_gram_indices, len(unique_words))
Y.extend(y_five_gram_onehot)

X.extend(word_seq_to_padded_embeddings(np.array(ten_gram_sequences)[:,:-1], word_index, 10))
y_ten_gram_indices = [word_index[word] for word in np.array(ten_gram_sequences)[:,-1]]
y_ten_gram_onehot = to_categorical(y_ten_gram_indices, len(unique_words))
Y.extend(y_ten_gram_onehot)

In [89]:
print(len(X))
print(len(Y))

382945
382945


### 2. Create 80/20 split for training and testing

In [90]:
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.1, random_state=40)

In [91]:
from torch.utils.data import TensorDataset, DataLoader

cross_entropy = torch.nn.CrossEntropyLoss()
train_data = TensorDataset(torch.from_numpy(np.array(X_train)), torch.from_numpy(np.array(Y_train)))
val_data = TensorDataset(torch.from_numpy(np.array(X_val)), torch.from_numpy(np.array(Y_val)))
test_data = TensorDataset(torch.from_numpy(np.array(X_test)), torch.from_numpy(np.array(Y_test)))

batch_size = 100

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)

# Define the Next word prediction LSTM model

In [92]:
class NextWordPredModel(torch.nn.Module):
    def __init__(self, embedding_dimension, hidden_units, num_classes):
        super().__init__()
        self.input_size = embedding_dimension
        self.hidden_units = hidden_units
        self.num_classes = num_classes
        self.n_layers=2
        
        self.rnn = torch.nn.LSTM(input_size = self.input_size, hidden_size=self.hidden_units,
                                 num_layers=self.n_layers, dropout=0.2, batch_first=True)
        self.fc = torch.nn.Linear(self.hidden_units, self.num_classes)
        
    def forward(self, x, state = None):
        h, rnn_state = self.rnn(x, state)
        h = self.fc(h)
        scores = F.log_softmax(h, dim=1)
        return scores[:,-1,:], rnn_state
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_units).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_units).zero_().to(device))
        return hidden

In [93]:
def train_model(model, epochs, lr=0.005):
    cross_entropy = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    model.train()
    for ep in tqdm(range(epochs)):
        init_state = model.init_hidden(batch_size)
        train_losses=[]
        num_correct = 0
        total = 0
        h = tuple([e.data for e in init_state])
        for idx, (inp_seq, op_word) in enumerate(train_loader):
            
            inp_seq, op_word = inp_seq.float().to(device), op_word.float().to(device)
            optimizer.zero_grad()

            prediction, (h_state, c_state) = model(inp_seq, h)
            h = (h_state.detach(), c_state.detach())
            
            loss = cross_entropy(prediction, op_word)
            num_correct += prediction.argmax(dim=1).eq(op_word.argmax(dim=1)).sum().item()
            total += op_word.shape[0]
            train_losses.append(loss.item())
    #         print("Loss = ", loss.item())
            loss.backward()
            optimizer.step()
        if (ep+1)%10 == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            val_num_correct = 0
            val_total = 0
            model.eval()
            val_h = tuple([each.data for each in val_h])
            
            for inp, lab in val_loader:
                
                inp, lab = inp.to(device), lab.to(device)
                out, (val_h_state, val_c_state) = model(inp.float(), val_h)
                val_h = (val_h_state.detach(), val_c_state.detach())
                
                val_num_correct += out.argmax(dim=1).eq(lab.argmax(dim=1)).sum().item()
                val_total += lab.shape[0]
                val_loss = cross_entropy(out, lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            if (num_correct >0):
                print("At least one correct :: num_correct = ", num_correct)
            train_acc = (num_correct / total)
            val_acc = (val_num_correct / val_total)
#             print(train_losses)
            print("Epoch: {}/{}...".format(ep+1, epochs),
                  "Step: {}...".format(idx),
                  "Train Loss: {:.6f}...".format(np.mean(train_losses)),
                  "Train Acc: {:.3f}...".format(train_acc),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "Val Acc: {:.3f}...".format(val_acc))

# Train the LSTM Model

In [427]:
# Training block for celebrity corpus
# Train Accuracy = 61.3% Val Accuracy = 33.2%
# model = NextWordPredModel(embedding_dimension=EMBEDDING_DIMENSION, hidden_units=256, num_classes=len(unique_words))
# train_model(model.to(device), 500, 0.0005)

  2%|▊                                       | 10/500 [02:55<2:26:53, 17.99s/it]

At least one correct :: num_correct =  33687
Epoch: 10/500... Step: 2298... Train Loss: 4.827836... Train Acc: 0.147... Val Loss: 5.064383 Val Acc: 0.148...


  4%|█▌                                      | 20/500 [05:54<2:26:23, 18.30s/it]

At least one correct :: num_correct =  54679
Epoch: 20/500... Step: 2298... Train Loss: 3.775284... Train Acc: 0.238... Val Loss: 4.421593 Val Acc: 0.202...


  6%|██▍                                     | 30/500 [08:54<2:24:02, 18.39s/it]

...

At least one correct :: num_correct =  140884
Epoch: 480/500... Step: 2298... Train Loss: 1.550345... Train Acc: 0.613... Val Loss: 4.600771 Val Acc: 0.331...


 98%|██████████████████████████████████████▏| 490/500 [2:27:16<03:04, 18.45s/it]

At least one correct :: num_correct =  140987
Epoch: 490/500... Step: 2298... Train Loss: 1.547027... Train Acc: 0.613... Val Loss: 4.597930 Val Acc: 0.333...


100%|███████████████████████████████████████| 500/500 [2:30:17<00:00, 18.04s/it]

At least one correct :: num_correct =  140968
Epoch: 500/500... Step: 2298... Train Loss: 1.543993... Train Acc: 0.613... Val Loss: 4.619086 Val Acc: 0.332...


In [94]:
# Training Block for Music corpus
model = NextWordPredModel(embedding_dimension=EMBEDDING_DIMENSION, hidden_units=256, num_classes=len(unique_words))
train_model(model.to(device), 500, 0.0005)

  2%|▊                                       | 10/500 [04:19<3:37:34, 26.64s/it]

At least one correct :: num_correct =  40176
Epoch: 10/500... Step: 3062... Train Loss: 5.045858... Train Acc: 0.131... Val Loss: 5.305742 Val Acc: 0.133...


  4%|█▌                                      | 20/500 [08:45<3:37:07, 27.14s/it]

At least one correct :: num_correct =  60930
Epoch: 20/500... Step: 3062... Train Loss: 4.159292... Train Acc: 0.199... Val Loss: 4.822711 Val Acc: 0.168...


  6%|██▍                                     | 30/500 [13:13<3:33:40, 27.28s/it]

...

At least one correct :: num_correct =  161122
Epoch: 480/500... Step: 3062... Train Loss: 2.026696... Train Acc: 0.526... Val Loss: 4.873804 Val Acc: 0.307...


 98%|██████████████████████████████████████▏| 490/500 [3:51:25<05:09, 30.92s/it]

At least one correct :: num_correct =  161804
Epoch: 490/500... Step: 3062... Train Loss: 2.015396... Train Acc: 0.528... Val Loss: 4.895746 Val Acc: 0.309...


100%|███████████████████████████████████████| 500/500 [3:56:31<00:00, 28.38s/it]

At least one correct :: num_correct =  161938
Epoch: 500/500... Step: 3062... Train Loss: 2.010200... Train Acc: 0.529... Val Loss: 4.889572 Val Acc: 0.308...


# Compute Test Accuracy

In [95]:
import math

test_losses = []
num_correct = 0
total = 0
h = model.init_hidden(batch_size)

model.eval()
h = tuple([each.data for each in h])
for inputs, labels in test_loader:
    
    inputs, labels = inputs.to(device), labels.to(device)
    output, (test_h_state, test_c_state) = model(inputs.float(), h)
    h = (test_h_state.detach(), test_c_state.detach())

    pred = output.argmax(dim=1)
    correct = pred.eq(labels.argmax(dim=1)).sum().item()

    num_correct += correct
    total += inputs.shape[0]
test_acc = round(num_correct/total, 3)
print("Test Accuracy = ", test_acc)

Test Accuracy =  0.309


# Save checkpoints for the trained models

In [96]:
from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d::%H:%M:%S")

PATH = 'model-' + keyword + "-" + str(len(X)) + "-" + now + "-" + str(test_acc) + ".pt"

torch.save({
            'model_state_dict': model.state_dict()
            }, PATH)
pickle.dump(word_embeddings, open("word_embeddings_" + keyword + "_" + now + ".pk", "wb"))

In [ ]:
input_sequence = word_seq_to_padded_embeddings(np.array(word_tokens), word_index, 5).unsqueeze(dim=1)

# LOAD TRAINED MODEL

In [33]:
MODEL_PATH = "model-287430-2022-11-21::10:22:40-0.322"
word_embeddings = pickle.load(open("word_embeddings_2022-11-21::10:22:40.pk", "rb"))

model = NextWordPredModel(embedding_dimension=EMBEDDING_DIMENSION, hidden_units=256, num_classes=len(unique_words))
checkpoint = torch.load(MODEL_PATH)

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

NextWordPredModel(
  (rnn): LSTM(64, 256, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=256, out_features=7901, bias=True)
)

# TEST THE TRAINED MODEL

In [97]:
def predict_next_word(word_model, sentence):
    words = sentence.strip().split()
    word_tokens = []
    for w in words:
        if w in word_index:
            word_tokens.append(word_index[w])
    
    input_sequence = word_embeddings[word_tokens].unsqueeze(dim=0)
    prediction, _ = word_model(input_sequence.to(device))
    predicted_wordindex = prediction.argmax(dim=1).item()
    predicted_word = index_word[predicted_wordindex]
    return predicted_word
    

In [98]:
processed_corpus[500:1000]

' styles of new mexico red dirt tejano and texas country its popularized roots originate in the southern and southwestern united states of the early 1920s country music often consists of ballads and honky tonk dance tunes with generally simple form folk lyrics and harmonies often accompanied by string instruments such as electric and acoustic guitars steel guitars such as pedal steels and dobros banjos and fiddles as well as harmonicas blues modes have been used extensively throughout its recorde'

In [100]:
pred_word = predict_next_word(model.to(device), "the celebrities arrived at the")
print("Next Word prediction = ", pred_word)

Next Word prediction =  ownership


In [109]:
pred_word = predict_next_word(model.to(device), "the guitar player was")
print("Next Word prediction = ", pred_word)

Next Word prediction =  spontaneously


In [111]:
pred_word = predict_next_word(model.to(device), "the song browser")
print("Next Word prediction = ", pred_word)

Next Word prediction =  depends


In [122]:
pred_word = predict_next_word(model.to(device), "rock music is rated very high in")
print("Next Word prediction = ", pred_word)

Next Word prediction =  opeth


In [134]:
sentence = "the main man was"
for i in range(10):
    predct = predict_next_word(model, sentence)
    sentence = sentence + " " + predct
print(sentence)

the main man was lyons deeper lighter frostbite maximum strong and minute hiatus hawaiian
